In [ ]:
!pip install Keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.8 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [ ]:
tf.__version__

'2.15.0'

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(train_image,train_labels), (test_images,test_labels) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [ ]:
#normalizing the train images
train_image = train_image/255.0
test_images = test_images/255.0

In [ ]:
train_image[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [ ]:
train_image[0].shape

(28, 28)

In [ ]:
#reshape the images
train_image = train_image.reshape(len(train_image),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [ ]:
##convulution layer
def model(hp):
  model=keras.Sequential([
      keras.layers.Conv2D(
          filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16), # No change
          kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
          activation='relu',
      ),keras.layers.Conv2D(
          filters=hp.Int('conv_2_filter',min_value=32,max_value=256,step=16), # Changed 'steps' to 'step'
          kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
          activation='relu',),keras.layers.Conv2D(
              filters=hp.Int('conv_3_filter',min_value=32,max_value=256,step=16), # Changed 'steps' to 'step'
          kernel_size=hp.Choice('conv_3_kernel',values=[3,5]),
          activation='relu',
          ),keras.layers.Flatten(), # Call the Flatten layer as a function
          keras.layers.Dense(
              units=hp.Int('dense_i_units',min_value=32,max_value=128,step=16), # No change
              activation='relu',
          ),
          keras.layers.Dense(10,activation='softmax')
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [ ]:
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-13-5147f5580bb4>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [ ]:
tuner_search=RandomSearch(model,
                               objective='val_accuracy',
                               max_trials=5,directory='output',project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_image,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 43s]
val_accuracy: 0.9051666855812073

Best val_accuracy So Far: 0.9076666831970215
Total elapsed time: 00h 05m 12s


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
# Assuming 'train_image' has shape (num_samples, height, width, channels)
model.build(input_shape=(None,)+ train_image.shape[1:])  # Build the model based on input shape
# Adding None to the input shape creates a batch dimension of any size

model.summary()  # Now you can print the summary

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  320       
                                                                 
 conv2d_1 (Conv2D)           multiple                  166608    
                                                                 
 conv2d_2 (Conv2D)           multiple                  915376    
                                                                 
 flatten (Flatten)           multiple                  0         
                                                                 
 dense (Dense)               multiple                  4562000   
                                                                 
 dense_1 (Dense)             multiple                  810       
                                                                 
Total params: 5645114 (21.53 MB)
Trainable params: 56451

In [ ]:
model.fit(train_image,train_labels,epochs=20,validation_split=0.1,initial_epoch=3)

Epoch 4/20
1688/1688 [==============================] - 26s 15ms/step - loss: 0.1687 - accuracy: 0.9366 - val_loss: 0.2537 - val_accuracy: 0.9120
Epoch 5/20
1688/1688 [==============================] - 25s 15ms/step - loss: 0.1319 - accuracy: 0.9504 - val_loss: 0.2687 - val_accuracy: 0.9127
Epoch 6/20
1688/1688 [==============================] - 25s 15ms/step - loss: 0.1023 - accuracy: 0.9619 - val_loss: 0.2938 - val_accuracy: 0.9130
Epoch 7/20
1688/1688 [==============================] - 25s 15ms/step - loss: 0.0778 - accuracy: 0.9712 - val_loss: 0.3088 - val_accuracy: 0.9168
Epoch 8/20
1688/1688 [==============================] - 25s 15ms/step - loss: 0.0640 - accuracy: 0.9763 - val_loss: 0.3360 - val_accuracy: 0.9135
Epoch 9/20
1688/1688 [==============================] - 25s 15ms/step - loss: 0.0533 - accuracy: 0.9805 - val_loss: 0.3704 - val_accuracy: 0.9117
Epoch 10/20
1688/1688 [==============================] - 25s 15ms/step - loss: 0.0454 - accuracy: 0.9834 - val_loss: 0.4299 

In [ ]:
model.evaluate(test_images,test_labels)

313/313 [==============================] - 2s 5ms/step - loss: 0.7471 - accuracy: 0.9110


[0.7471396327018738, 0.9110000133514404]